# Estimate diffusivity parameters from 1D b-space optimization
## Ryan Holmes

In [ ]:
# Load modules
import numpy as np
import h5py
import matplotlib
import shutil
import os
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
matplotlib.rcParams["figure.facecolor"] = "white"
matplotlib.rcParams["axes.facecolor"] = "white"
matplotlib.rcParams["savefig.facecolor"] = "white"
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import time
from IPython import display
import obsfit1d
from scipy.interpolate import interp1d
import bbin

In [ ]:
# Do a single run for checks:
# basedir = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns24-5-19/';
# basedir = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns_wide30-5-19/';
basedir = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns_bIC30-5-19/';
# run = basedir + 'z0_0p1250_AH_000_Kinf_m5_ADV_2_slope_400/'#_4dt_2dz/'
run = basedir + 'ADV_2_AH_010_slope_200_maxy_10000000_miny_600/'

# Get observed data:
params = np.load(run + 'runparams.npz');locals().update(params)

with h5py.File(run + 'ifields.h5', mode='r') as file:
    y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];

#tinds = np.arange(0,100,10)
#tinds = range(0,100,50)
tinds = range(0,50)

with h5py.File(run + 'snapshots.h5', mode='r') as file:

    t = file['scales/sim_time'][:]/lday;
    trT = file['tasks']['trT'][:,0,0]
    tr = file['tasks']['tr'][0,:,:]
    b, trC = bbin.trbinI(y,z,tr,N2,theta,Lz,Ly,q0)
    trF = np.zeros((len(b),len(t)))
    for i in tinds:#len(t),pfr):
        if ((i % 10) == 0):
            print('Doing b-profile time %03d of %03d, run %s' % (i, tinds[-1], run))
        tr = file['tasks']['tr'][i,:,:]
        b, trF[:,i] = bbin.trbinI(y,z,tr,N2,theta,Lz,Ly,q0)
        

zF = b/N2;
dzF = zF[1]-zF[0]
imax = np.argmax(trF[:,0])
zF = zF - zF[imax]

# Do 3-parameter fit:
#tinds = range(0,len(t))
K = [];Kh = [];w = [];func = [];suc = [];
Kerr = []; Kherr = []; werr = [];msg = [];
for ti in tinds:
    if ((ti % 10) == 0):
        print('Doing time %03d of %03d, run %s' % (ti, tinds[-1], run))
    res = obsfit1d.fit3par(zF,trF[:,ti],trF[:,0],int(t[ti]*1e5))
    obsfit1d.plot(res.x,zF,trF[:,ti],trF[:,0],int(t[ti]*1e5))
    K.append(res.x[0])
    Kh.append(res.x[1])
    w.append(res.x[2])
    Kerr.append(res.xerr[0])
    Kherr.append(res.xerr[1])
    werr.append(res.xerr[2])
    suc.append(res.success)
    func.append(res.fun)
    msg.append(res.msg)

K = np.array(K)
Kerr = np.array(Kerr)
Kh = np.array(Kh)
Kherr = np.array(Kherr)

f = plt.figure(figsize=(10,10),facecolor='white')
plt.subplot(2,1,1)
plt.plot(tinds,Kerr/K,'xk')
#cor = [x <= 0.00001 for x in func]
#incor = [x > 0.00001 for x in func]
plt.subplot(2,1,2)
tinds = np.array(tinds);
plt.plot(t[tinds],-K,'xk');
plt.plot(t[tinds],-(K+2*Kerr),'xr')
plt.plot(t[tinds],-(K-2*Kerr),'xb')


In [ ]:
# Process multiple runs:
basedir = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns_layer30-08-18/';
#basedir = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns24-08-18/';
# folds = [basedir + x + '/' for x in os.listdir(basedir)]
# folds = folds[-10:]
#folds = [basedir + 'AH_%03d_ADV_2_Kinf_m5_mny0_0p5333_slope_200_isoAH/' % x for x in [0.,10.,20.,30.,40.,50.,60.,70.,80.,90.]]#,125,175,200)]
folds = [basedir + 'AH_%03d_ADV_2_Kinf_m5_mny0_0p5333_slope_200_isoAH_Lz4000/' % x for x in [100.,125.,175.,200.]]
# folds = [basedir + 'AH_%03d_ADV_2_Kinf_m5_mny0_0p4667_slope_200_isoAH/' % x for x in [10.]]#,20,30,40,50,60,70,80,90,125,175,200)]
folds

tfin = 101;

for run in folds:

    # Get observed data:
    params = np.load(run + 'runparams.npz');locals().update(params)

    with h5py.File(run + 'ifields.h5', mode='r') as file:
        y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];

    with h5py.File(run + 'snapshots.h5', mode='r') as file:

        t = file['scales/sim_time'][:]/lday;
        trT = file['tasks']['trT'][:,0,0]
        tr = file['tasks']['tr'][0,:,:]
        b, trC = bbin.trbinI(y,z,tr,N2,theta,Lz,Ly,q0,Kinf,K0,d,SPru0i)
        trF = np.zeros((len(b),tfin))
        for i in range(0,tfin):#len(t),pfr):
            if ((i % 10) == 0):
                print('Doing b-profile time %03d of %03d, run %s' % (i, tfin, run))
            tr = file['tasks']['tr'][i,:,:]
            b, trF[:,i] = bbin.trbinI(y,z,tr,N2,theta,Lz,Ly,q0,Kinf,K0,d,SPru0i)

    zF = b/N2;
    dzF = zF[1]-zF[0]
    imax = np.argmax(trF[:,0])
    zF = zF - zF[imax]

    # Do 3-parameter fit:
    tinds = range(0,tfin)
    K = [];Kh = [];w = [];func = [];suc = [];
    Kerr = []; Kherr = []; werr = [];
    for ti in tinds:
        if ((ti % 10) == 0):
            print('Doing time %03d of %03d, run %s' % (ti, tfin, run))
        res = obsfit1d.fit3par(zF,trF[:,ti],trF[:,0],int(t[ti]*1e5))
        # obsfit1d.plot(res.x,zF,trF[:,ti],trF[:,0],int(t[ti]*1e5))
        K.append(res.x[0])
        Kh.append(res.x[1])
        w.append(res.x[2])
        Kerr.append(res.xerr[0])
        Kherr.append(res.xerr[1])
        werr.append(res.xerr[2])
        suc.append(res.success)
        func.append(res.fun)

    # Save to file:
    np.savez(run + 'par3fitGOODINI',K=K,Kh=Kh,w=w,func=func,suc=suc,Kerr=Kerr,Kherr=Kherr,werr=werr,t = t[tinds])